# Ichi Told Me

### Assumptions:

It is difficult to catch every top and bottom of a market.
>That is where the smart money is shaking out the weak hands.           https://www.youtube.com/watch?v=TzKQwttv9IA

Ichimoku is a full fleged trading system.
>The cloud system is designed to capturing 80% of price movements (I.e. 80/20 rule)
https://www.linkedin.com/pulse/ichimoku-cloud-jonathan-camphin/

Rebalancing preforms ~63% better than HODL on trending and ranging markets.

AND
>Rebalancing once every hour resulted in a median portfolio performance boost of 94% over buy-and-hold. 83.0% of all portfolios which used hourly rebalances performed better than buy-and-hold.
https://blog.shrimpy.io/blog/cryptocurrency-portfolio-rebalancing-bittrex-analysis

AND

>A 15% threshold rebalance outperformed HODL by 305%.
https://blog.shrimpy.io/blog/the-best-threshold-for-cryptocurrency-rebalancing-strategies

### Goal:

__Identify strong to medium Ichimoku signals on a 1 hour timeframe to create dynamic portfolios that can be rebalanced hourly or at a percent deviation threshold on https://www.shrimpy.io/.__

### Tools:

#### Languages: 
Python

#### Visuals:
DASH by Plotly https://dash.plot.ly

#### APIs: 
Shrimpy Developer API  https://developers.shrimpy.io/docs/#introduction

#### Deployment:
[DigitalOcean](https://www.digitalocean.com/) or [Vultr](https://www.vultr.com/) servers.

____

## RULES

These rules are more conservative than most and are designed to nearly eliminate false
signals.

https://www.youtube.com/watch?v=X5lxBTrN-Yk&t=1318s

    CLOSE ABOVE CLOUD
    Close > Kijun_Sen & Close > Tenkan_Sen 
    
    CONVERSION LINE IS GREATER THAN BASE LINE
    Tenkan_Sen > Kijun_Sen
    
    *KEY
    CLOUD AHEAD IS BULLISH (GREEN) *look displacement ahead
    cloud.shift(displacement) == green 
    
    *KEY
    LAGGIN SPAN IS GREATER OR CROSSES ABOVE CLOUD  *look displacement behind
    Chikou_Span > cloud.shift(-displacement)

____

In [1]:
# imports
import numpy as np
from pandas import DataFrame, Series
import math

import json
import requests
import pandas as pd
from pandas.io.json import json_normalize

import decimal
from datetime import datetime, timedelta
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
from mpl_finance import candlestick_ohlc
import matplotlib.pyplot as plt

## Get Supported Exchanges
Only required when scaling to other exchanges.

In [2]:
# HTTP Request
# GET https://dev-api.shrimpy.io/v1/list_exchanges

def list_exchanges():
    
    """This endpoint retrieves all Shrimpy supported exchanges
    and some basic information about each.
    
    :return: pandas.DataFrame
    """
    res = requests.get("https://dev-api.shrimpy.io/v1/list_exchanges")
    return json_normalize(res.json())

In [3]:
list_exchanges().head()

,bestCaseFee,exchange,icon,worstCaseFee
0,0.000150,binance,https://assets.shrimpy.io/exchanges/binance.png,0.0010
1,0.002500,bittrex,https://assets.shrimpy.io/exchanges/bittrex.png,0.0025
2,0.002500,bittrexinternational,https://assets.shrimpy.io/exchanges/bittrexint...,0.0025
3,0.000125,kucoin,https://assets.shrimpy.io/exchanges/kucoin.png,0.0010
4,0.000000,coinbasepro,https://assets.shrimpy.io/exchanges/coinbasepr...,0.0030


## Get Trading Pairs

Pick and exchange to start with.
Binance is popular and has a lot of trading volume and pairs.
However, Binance like someother exchanges, will discontinue service to US customers soon.
https://www.theblockcrypto.com/2019/06/14/us-customers-to-be-blocked-from-trading-on-binance-com/

In [4]:
# HTTP Request
# GET https://dev-api.shrimpy.io/v1/exchanges/<exchange>/trading_pairs

def trading_pairs(exchange):
    
    """This endpoint retrieves a list of active trading pairs for a particular exchange.
    The symbols will match the tradingSymbol from Get Exchange Assets as well as the
    symbol used by the exchange.
    
    :param exchange: string ex: 'Bittrex', 'Coinbase'
    :return: pandas.DataFrame
    """
    response = requests.get(f"https://dev-api.shrimpy.io/v1/exchanges/{exchange}/trading_pairs")
    return json_normalize(response.json())

In [5]:
trading_pairs('binance').head()

,baseTradingSymbol,quoteTradingSymbol
0,ETH,BTC
1,LTC,BTC
2,BNB,BTC
3,NEO,BTC
4,QTUM,ETH


## Get Candles

In [6]:
# HTTP Request
# GET https://dev-api.shrimpy.io/v1/exchanges/<exchange>/candles

def candles(exchange, quoteTradingSymbol, baseTradingSymbol, interval):
    
    """This endpoint retrieves live candlestick data.

    When retrieving candlestick data for plotting, first call the endpoint without 
    specifying a startTime. This will return data associated with the most recent 
    1000 candlesticks. Subsequently, periodically call the endpoint specifying the 
    startTime as the time associated with the most recent candlestick. Note that the 
    last or most recent candlestick is for the current, not-yet-committed frame.
    
    :param exchange: string: The exchange for which to retrieve candlestick data.
    :param baseTradingSymbol: string: The base symbol of a pair on the exchange. 
        (e.g. XLM for a XLM-BTC pair)
    :param quoteTradingSymbol: string: The quote symbol of a pair on the exchange. 
        (e.g. BTC for a XLM-BTC pair)
    :param interval: string: The interval must be one of the following values: 
        1m, 5m, 15m, 1h, 6h, or 1d) These values correspond to intervals representing 
        one minute, five minutes, fifteen minutes, one hour, six hours, and one day, 
        respectively.
    :param startTime (optional): Date Optionally only return data on or after the 
        supplied startTime (inclusive).
    
    :return: pandas.DataFrame
    
    """
    
    response = requests.get(f"https://dev-api.shrimpy.io/v1/exchanges/{exchange}/candles?quoteTradingSymbol={quoteTradingSymbol}&baseTradingSymbol={baseTradingSymbol}&interval={interval}")
    return json_normalize(response.json())

In [44]:
df = candles('binance', 'USDT', 'BTC', '1h')

In [45]:
df.head()

,btcVolume,close,high,low,open,quoteVolume,time,usdVolume,volume
0,816.491962,7966.6600000000000000,7996.0000000000000000,7934.3600000000000000,7964.8600000000000000,6.506649e+06,2019-05-25T00:00:00.000Z,6.506649e+06,816.5493680000000000
1,814.145003,8005.0200000000000000,8033.4800000000000000,7957.0600000000000000,7968.9400000000000000,6.516023e+06,2019-05-25T01:00:00.000Z,6.516023e+06,814.1089040000000000
2,526.264495,8009.9800000000000000,8012.4400000000000000,7970.6700000000000000,8004.6800000000000000,4.207625e+06,2019-05-25T02:00:00.000Z,4.207625e+06,526.2805760000000000
3,697.951093,7975.2900000000000000,8021.5400000000000000,7965.0000000000000000,8006.8800000000000000,5.577362e+06,2019-05-25T03:00:00.000Z,5.577362e+06,698.0021610000000000
4,516.521013,7985.8400000000000000,8013.8100000000000000,7972.7600000000000000,7976.9600000000000000,4.131324e+06,2019-05-25T04:00:00.000Z,4.131324e+06,516.5090330000000000


## Create Ichimoku data points

Unique to the Ichimoku system is that part of the signal is projected ahead in time.

In [46]:
def calc_ichimoku(df, settings = 'crypto'): #, to_plot= False
    
    """Calculates ichimoku cloud data points
    
    :param df: pandas.DataFrame: 
    :param settings: string: 
    
    :return: pandas.DataFrame
    
    """
    
#     set setting
    if settings == 'double':
        conversion_line_period = 40
        base_line_period = 60
        lagging_span_period = 120
        displacement_period = base_line_period
        
    elif settings == 'standard':
        conversion_line_period = 9
        base_line_period = 26
        lagging_span_period = 52
        displacement_period = base_line_period

    else :
        conversion_line_period = 20
        base_line_period = 30
        lagging_span_period = 60
        displacement_period = base_line_period
        
#     Calculate tenkan_sen
#     df['tenkan_sen'] = (HighPriceMax(H, conversion_line_period) + LowPriceMin(L, conversion_line_period)) / 2
    
    conversion_high = df['high'].rolling(window= conversion_line_period).max()
    conversion_low = df['low'].rolling(window= conversion_line_period).min()
    df['tenkan_sen'] = (conversion_high + conversion_low) /2
    
#     Calculate kijun_sen
#     df['kijun_sen'] = (HighPriceMax(H, base_line_period) + LowPriceMin(L, base_line_period)) / 2
    
    base_high = df['high'].rolling(window= base_line_period).max()
    base_low = df['low'].rolling(window= base_line_period).min()
    df['kijun_sen'] = (base_high + base_low) /2


#     extend df into future displacement_period
    last_index = df.iloc[-1:].index[0]
    last_date = pd.to_datetime(df['time'],utc=True).iloc[-1]
    for i in range(1,displacement_period + 1):
        df.loc[last_index+1 +i, 'time'] = last_date + timedelta(hours=i)
        
#     calculate senkou_span_a
    df['senkou_span_a'] = ((df['tenkan_sen'] + df['kijun_sen']) / 2).shift(displacement_period)

#     calculate senkou_span_b
#     df['senkou_span_b'] = ((HighPriceMax(H, lagging_span_period) + LowPriceMin(L, lagging_span_period)) / 2).shift(displacement_period)

    lag_high = df['high'].rolling(window= lagging_span_period).max()
    lag_low = df['low'].rolling(window= lagging_span_period).min()
    df['senkou_span_b'] = ((lag_high + lag_low) /2).shift(displacement_period)
    
    
#     calulate chikou_span
    df['chikou_span'] = df['close'].shift(-displacement_period).apply(pd.to_numeric, errors='coerce') #sometimes -26 

    # TODO
    #     Edit fill to represent bullish (green) and bearish (red) clouds 
    #     between the leading spans.
    
#   tmp = df[['time','close','senkou_span_a','senkou_span_b','kijun_sen','tenkan_sen', 'chikou_span']]
    
    df['time'] = pd.to_datetime(df['time'],utc=True)
    df['close'] = df['close'].apply(pd.to_numeric, errors='coerce')
    
#     tmp = df[['time','close','senkou_span_a','senkou_span_b','kijun_sen','tenkan_sen', 'chikou_span']].tail(300)
    
#     tmp.index = tmp['time']
#     del tmp['time']
    
#     a1 = tmp.plot(figsize=(15,10))
#     a1.fill_between(tmp.index, tmp.senkou_span_a, tmp.senkou_span_b);
    
    return df


In [47]:
# calc_ichimoku(df)

## Set Rules

In [27]:
# CLOSE ABOVE CLOUD
# Close > Kijun_Sen & Close > Tenkan_Sen

df['close_above_cloud'] = np.where((df['close'] > df['kijun_sen']) & (df['close'] > df['tenkan_sen']), 1, 0)

# CONVERSION LINE IS GREATER THAN BASE LINE
# Tenkan_Sen > Kijun_Sen

df['t_above_k'] = np.where((df['tenkan_sen'] > df['kijun_sen']), 1, 0)


# *KEY
# CLOUD AHEAD IS BULLISH (GREEN) *look displacement ahead
# cloud.shift(displacement) == green 

df['green_cloud_ahead'] = np.where((df['senkou_span_a'] > df['senkou_span_b']).shift(-60).apply(pd.to_numeric, downcast='integer'), 1, 0)

# *KEY
# LAGGIN SPAN IS GREATER OR CROSSES ABOVE CLOUD  *look displacement behind
# Chikou_Span > cloud.shift(-displacement)

df['chikou_span_breaks_cloud'] = np.where((df['chikou_span'].shift(30) > df['senkou_span_a'].shift(30)) & (df['chikou_span'].shift(30) > df['senkou_span_b'].shift(30)).apply(pd.to_numeric, downcast='integer'), 1, 0)

# create trade condition feature
df['trade'] = np.where((df['close_above_cloud'] == 1) & (df['t_above_k'] == 1) & (df['green_cloud_ahead'] == 1) & (df['chikou_span_breaks_cloud'] == 1), 1, 0)

In [39]:
# df[-60:-20]

In [40]:
#     tmp = df[['time','close','senkou_span_a','senkou_span_b','kijun_sen','tenkan_sen', 'chikou_span','trade']].tail(300)
    
#     tmp.index = tmp['time']
#     del tmp['time']
    
#     a1 = tmp.plot(figsize=(15,10))
#     a1.fill_between(tmp.index, tmp.senkou_span_a, tmp.senkou_span_b);
    
    

____
____
# TODO 

#### (move this cell to above the next step to be completed)

____
____

## Create portfolio

In [15]:
# Check ichi signals on the 1 hour timeframe.
# Create a list of coins to be included.

def create_portfolio(exchange, interval):   #TODO
                                            # interval set to 1h unless 
                                            # MAXIMUM_PORTFOLIO_SIZE has been reached,
                                            # then resample to 4h. 
    portfolio = []
    df = {}
    
    coinlist = ticker(exchange).symbol.tolist()
    print('Checking on ' + str(len(coinlist)) + ' coins.')
    
    for coin in coinlist:
        
        try:
            
            df = check_coin(exchange, coin, interval)
            
            if df['condition'][-1:].values == 'buy':
                portfolio.append(coin)
                print('Added ' + coin + ' to portfolio.')
            else: 
                pass

        except:
            print(coin + ' is not availalbe.')
        
#         if df['condition'][-1:].values == 'buy':
#             portfolio.append(coin)
#             print('Added ' + coin + ' to portfolio.')

    return portfolio

## Optimize portfolio

The plan here is to maximize the portolio proformance based on the probability that a 
stong trend will continue.

Do this by checking the ichi signals on a second (Higher) timeframes and sorting by 
those that are on both timeframes and truncate the list at the `MAXIMUM_PORTFOLIO_SIZE`.

The coins that are on both timeframes with be at the top of the list while the others will be lower.

In [16]:
# Could combine or included in 'create_portfolio' function.

def optimize_portfolio(portfolio):
    print(f"Portfolio has {len(portfolio)} coins in it.")
    if len(portfolio) >= MAXIMUM_PORTFOLIO_SIZE + 1:
        # create_portfolio(exchange, '4h')
        print("That's too many")

## Distribute Portfolio Allocations
Could combine or included in `create_portfolio` function.


In [17]:
# Distributes coins evenly and takes into account 'BNB' coin to reduce trade fees.

def distribute_allocations(coins):
    
    # create empty list
    allocations = []
    coins_plus = []
    
    # remove duplicates and turn all coins to uppercase
    coins = list(dict.fromkeys([x.upper() for x in coins]))
    
    #TODO
    
    # if len(coins) > coin_limit:
    #   coins = limit_coins(coins)
        
    
    # code if 'BNB' is in the list
    if 'BNB' in coins:
        a = round(100/len(coins),2)
        r = round(100 - a * len(coins),2)
    
        # loop through the coins in the list
        for i in coins:
            
            # Special rules for 'BNB'
            if 'BNB' in coins:
                if i == 'BNB':
                    b = {"symbol" : i , "percent" : round((a + r),2) }
                    allocations.append(b)

                else:    
                    b = {"symbol" : i , "percent" : a }
                    allocations.append(b)

    # code if 'BNB' is NOT in the list
    if 'BNB' not in coins:
#         a = round(100/len(coins),2)
#         r = round(100 - a * len(coins),2)
        
        if r == 0:
            coins_plus = coins
            coinscoins_plus.append('BNB')
        
#         if r == 0:
#             a = round(97/len(coins),2)
#             r = round(97 - a * len(coins),2)
        
        # add 'r' amount of 'BNB' to list before looping
        allocations.append({"symbol" : 'BNB' , "percent" : r + 3})
        
        # loop through the coins in the list
        for i in coins_plus:
            b = {"symbol" : i , "percent" : a }
            allocations.append(b) 

    print(a*len(coins)+r, r)
          
    return allocations
    
    #if remander is not 0:
        

____

# Sending it all to Shrimpy

## Creating and Signing a Request
https://developers.shrimpy.io/docs/#creating-a-request

https://developers.shrimpy.io/docs/#signing-a-request

In [18]:
# # snipet from shrimpy reddit
# https://www.reddit.com/r/ShrimpyApp/comments/be6o26/shrimpy_api_python_36/
    
import requests, json, time, datetime, base64, hmac, hashlib

key = 'keykeykeykeykeykeykeykeykeykeykeykey'
secret = 'secretsecretsecretsecretsecretsecretsecretsecretsecretsecretsecretsecretsecretsecretsecretsecretsecret'
nonce = str(int(time.time()))

endpoint = '/v1/accounts/'

signurl = (endpoint + 'GET'+nonce).encode('utf-8')

signing = hmac.new(base64.b64decode(secret), signurl , hashlib.sha256)

signing_b64 = base64.b64encode(signing.digest()).decode('utf-8')

header = {'content-type': 'application/json', 
          'SHRIMPY-API-KEY': key,
            'SHRIMPY-API-NONCE' : nonce,
            'SHRIMPY-API-SIGNATURE': signing_b64 }


r = requests.get('https://api.shrimpy.io' + endpoint, headers = header)
print(r)
print(r.text)

Error: Incorrect padding

____

# See if it works.

## Build Backtest

To run a backtest on each iteration of the portolios (changing often), a list of 
optimized porfolios must be created with `startTime` and `endTime` values.
Use the last `usdValue` as the `initialValue` for the next iteration.

In [49]:
# Request Body

{
    "rebalancePeriod": 24,
    "fee": "0.1",
    "startTime": "2018-05-19T00:00:00.000Z",
    "endTime": "2018-11-02T00:00:00.000Z",
    "initialValue": "5000.0",
    "allocations": [
        { "symbol": "BTC", "percent": "50.0" },
        { "symbol": "ETH", "percent": "50.0" }
    ]
}

{'rebalancePeriod': 24,
 'fee': '0.1',
 'startTime': '2018-05-19T00:00:00.000Z',
 'endTime': '2018-11-02T00:00:00.000Z',
 'initialValue': '5000.0',
 'allocations': [{'symbol': 'BTC', 'percent': '50.0'},
  {'symbol': 'ETH', 'percent': '50.0'}]}

## Run Backtest
This endpoint runs a backtest with the supplied settings. 
This endpoint has a rate limit of __20 requests per minute__.

In [ ]:
# HTTP Request
# POST https://dev-api.shrimpy.io/v1/analytics/backtest/<exchange>/run

____

# Next

##  Set the Strategy

This endpoint sets the strategy for the exchange account. All future rebalance operations will use the new strategy. 

`portfolio = allocations` in request body

In [ ]:
# HTTP Request
# POST https://dev-api.shrimpy.io/v1/users/<userId>/accounts/<exchangeAccountId>/strategy

## Set the Rebalance Period
This endpoint sets the rebalance period for the specified exchange account. The next rebalance will be scheduled from the current time. For example, it is currently 4PM and a rebalance period of 24 hours is set, the next rebalance operation will occur at 4PM tomorrow.

In [ ]:
# HTTP Request
# POST https://dev-api.shrimpy.io/v1/users/<userId>/accounts/<exchangeAccountId>/rebalance_period

# Set to 1h, but reset to makes sure the proper rebalance period is active.


## Queue Rebalance
This endpoint queues a rebalance for the specified exchange account. When the rebalance occurs, trades will be made so that the linked exchange accounts assets match the current strategy. That is, any assets that are overallocated will be sold and any assets that are underallocated will be purchased.

In [ ]:
# HTTP Request
# POST https://dev-api.shrimpy.io/v1/users/<userId>/accounts/<exchangeAccountId>/rebalance

## Get Balance
This endpoint retrieves detailed balance data for an exchange account. By default, the most recent balance is returned. Balance is retrieved from the exchange every 15 minutes for each account, as well as immediately after rebalance operations and trade operations.

In [ ]:
# HTTP Request
# GET https://dev-api.shrimpy.io/v1/users/<userId>/accounts/<exchangeAccountId>/balance

____

# Additional features / thoughts

## Sorting portfolio

    Consider another metric to sort the optimized portfolio.
        - OBV On by Volume
        - ATR Average true Range
        - ADX Average Directional Index
        - Stochastics
        - **other**
        
    When to quote in USD (or stable coin) vs. BTC
        if BTC_uptrend:
         quote = USDT
        elif BTC_downtend:
         quote = BTC
        else :
         quote = [“USDT”, “BTC”]
         
        Does it really matter?

## Notifiers

#### Activity

    channels:
       - text message 
       - discord 
       - telegram
       
    include: (per exchange)
        - coins added
        - coins removed
        - current balance
        - balance change (USD and BTC percent and quantity)
            - 1 day
            - 1 week
            - 1 month
            - 6 months
            - 1 year
            - All time
        - link to dashboard
    
#### Social

    channels:
        - Twitter
        - Instagram
        
    include:
        - select high proforming coin from last portfolio
        - image of trade
        
    referal links: ?
        - follow via shrimpy
        - sign up at exchange
      
#### System monitors
    
    channels:
       - text message
       
    include:
        - error
        - time